# Module 4.7: HOG + SVM 影像分類器（專案整合）

## 學習目標

這是 Module 4 的整合專案。完成後你將能夠：

1. 結合 HOG 特徵擷取與 SVM 分類器
2. 建立完整的影像分類 pipeline
3. 評估分類器效能（準確率、混淆矩陣）
4. 理解傳統機器學習在電腦視覺中的應用

## 背景知識

HOG + SVM 是經典的物件偵測方法，Dalal & Triggs (2005) 最初用於行人偵測。

**Pipeline**：
1. 影像預處理（調整大小、灰階化）
2. HOG 特徵擷取
3. SVM 分類

---

## 參考論文

- Dalal & Triggs, *"Histograms of Oriented Gradients for Human Detection"*, CVPR 2005

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

plt.rcParams['figure.figsize'] = [10, 6]
np.random.seed(42)

print("環境設定完成！")

---

## Part 1: HOG 特徵擷取器（從 Module 3 複製）

我們需要使用之前實作的 HOG 特徵擷取器。

In [ ]:
# HOG 實作（從 Module 3 移植）

def compute_gradients(image):
    """
    計算圖像的梯度大小和方向
    """
    H, W = image.shape
    img = image.astype(np.float64)
    
    # Gx
    gx = np.zeros((H, W), dtype=np.float64)
    gx[:, 1:-1] = img[:, 2:] - img[:, :-2]
    gx[:, 0] = img[:, 1] - img[:, 0]
    gx[:, -1] = img[:, -1] - img[:, -2]
    
    # Gy
    gy = np.zeros((H, W), dtype=np.float64)
    gy[1:-1, :] = img[2:, :] - img[:-2, :]
    gy[0, :] = img[1, :] - img[0, :]
    gy[-1, :] = img[-1, :] - img[-2, :]
    
    magnitude = np.sqrt(gx**2 + gy**2)
    direction = np.degrees(np.arctan2(gy, gx)) % 180
    
    return magnitude, direction


class HOGDescriptor:
    """
    HOG 特徵描述子
    """
    
    def __init__(self, cell_size=8, block_size=2, num_bins=9):
        self.cell_size = cell_size
        self.block_size = block_size
        self.num_bins = num_bins
        self.block_dim = block_size * block_size * num_bins
    
    def _compute_cell_histogram(self, magnitude, direction):
        """計算 cell 直方圖（soft binning）"""
        bin_width = 180.0 / self.num_bins
        bin_pos = direction / bin_width
        
        bin_left = np.floor(bin_pos).astype(int) % self.num_bins
        bin_right = (bin_left + 1) % self.num_bins
        weight_right = bin_pos - np.floor(bin_pos)
        weight_left = 1 - weight_right
        
        hist = np.zeros(self.num_bins)
        np.add.at(hist, bin_left.flatten(), (magnitude * weight_left).flatten())
        np.add.at(hist, bin_right.flatten(), (magnitude * weight_right).flatten())
        
        return hist
    
    def _normalize_block(self, block, eps=1e-5):
        """L2-Hys 正規化"""
        norm = np.sqrt(np.sum(block**2) + eps)
        normalized = block / norm
        normalized = np.minimum(normalized, 0.2)
        norm2 = np.sqrt(np.sum(normalized**2) + eps)
        return normalized / norm2
    
    def compute(self, image):
        """
        計算 HOG 特徵
        
        Parameters
        ----------
        image : np.ndarray
            灰階圖像，shape (H, W)
        
        Returns
        -------
        features : np.ndarray
            HOG 特徵向量
        """
        H, W = image.shape
        num_cells_y = H // self.cell_size
        num_cells_x = W // self.cell_size
        
        # 計算梯度
        magnitude, direction = compute_gradients(image)
        
        # 計算 cell 直方圖
        cell_hists = np.zeros((num_cells_y, num_cells_x, self.num_bins))
        
        for cy in range(num_cells_y):
            for cx in range(num_cells_x):
                y_start = cy * self.cell_size
                x_start = cx * self.cell_size
                
                cell_mag = magnitude[y_start:y_start+self.cell_size,
                                    x_start:x_start+self.cell_size]
                cell_dir = direction[y_start:y_start+self.cell_size,
                                    x_start:x_start+self.cell_size]
                
                cell_hists[cy, cx] = self._compute_cell_histogram(cell_mag, cell_dir)
        
        # Block normalization
        num_blocks_y = num_cells_y - self.block_size + 1
        num_blocks_x = num_cells_x - self.block_size + 1
        
        features = []
        
        for by in range(num_blocks_y):
            for bx in range(num_blocks_x):
                block = cell_hists[by:by+self.block_size,
                                  bx:bx+self.block_size].flatten()
                features.append(self._normalize_block(block))
        
        return np.concatenate(features)
    
    def get_feature_dim(self, height, width):
        """計算特徵維度"""
        num_cells_y = height // self.cell_size
        num_cells_x = width // self.cell_size
        num_blocks_y = num_cells_y - self.block_size + 1
        num_blocks_x = num_cells_x - self.block_size + 1
        return num_blocks_y * num_blocks_x * self.block_dim


# 測試 HOG
print("=== 測試 HOG ===")

hog = HOGDescriptor(cell_size=8, block_size=2, num_bins=9)

# 測試圖像
test_img = np.random.rand(64, 64) * 255
features = hog.compute(test_img)

print(f"圖像大小: 64x64")
print(f"HOG 特徵維度: {len(features)}")
print(f"預期維度: {hog.get_feature_dim(64, 64)}")

---

## Part 2: Linear SVM（從 Module 4.4 複製）

In [ ]:
# Linear SVM 實作

class LinearSVM:
    """
    Linear SVM with Hinge Loss
    """
    
    def __init__(self, C=1.0):
        self.C = C
        self.reg = 1.0 / C
        self.w = None
        self.b = None
    
    def fit(self, X, y, lr=0.001, n_iter=1000, verbose=False):
        """
        訓練 SVM
        
        Parameters
        ----------
        X : np.ndarray, shape (N, D)
        y : np.ndarray, shape (N,)
            標籤為 -1 或 +1
        """
        N, D = X.shape
        y = np.where(y == 0, -1, y)
        
        self.w = np.zeros(D)
        self.b = 0.0
        
        for i in range(n_iter):
            scores = X @ self.w + self.b
            margins = y * scores
            violating = (margins < 1).astype(float)
            
            dw = -(1/N) * (X.T @ (violating * y)) + self.reg * self.w
            db = -(1/N) * np.sum(violating * y)
            
            self.w = self.w - lr * dw
            self.b = self.b - lr * db
            
            if verbose and (i + 1) % (n_iter // 10) == 0:
                hinge = np.mean(np.maximum(0, 1 - margins))
                print(f"Iter {i+1}: Hinge Loss = {hinge:.4f}")
        
        return self
    
    def predict(self, X):
        """預測標籤（-1 或 +1）"""
        scores = X @ self.w + self.b
        return np.sign(scores)
    
    def decision_function(self, X):
        """返回決策分數"""
        return X @ self.w + self.b
    
    def score(self, X, y):
        """計算準確率"""
        y = np.where(y == 0, -1, y)
        predictions = self.predict(X)
        return np.mean(predictions == y)


# Multi-class SVM (One-vs-All)
class MultiClassSVM:
    """
    使用 One-vs-All 的多分類 SVM
    """
    
    def __init__(self, n_classes, C=1.0):
        self.n_classes = n_classes
        self.C = C
        self.classifiers = []
    
    def fit(self, X, y, lr=0.001, n_iter=1000, verbose=False):
        self.classifiers = []
        
        for k in range(self.n_classes):
            if verbose:
                print(f"訓練分類器 {k}...")
            
            y_binary = np.where(y == k, 1, -1)
            
            svm = LinearSVM(C=self.C)
            svm.fit(X, y_binary, lr=lr, n_iter=n_iter, verbose=False)
            
            self.classifiers.append(svm)
        
        return self
    
    def predict(self, X):
        scores = np.zeros((X.shape[0], self.n_classes))
        
        for k, svm in enumerate(self.classifiers):
            scores[:, k] = svm.decision_function(X)
        
        return np.argmax(scores, axis=1)
    
    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions == y)


print("SVM 類別載入完成！")

---

## Part 3: 建立合成資料集

為了測試，我們建立一個簡單的合成資料集：不同形狀的圖像。

In [ ]:
# 建立合成圖像資料集

def create_circle_image(size=64):
    """建立圓形圖像"""
    img = np.ones((size, size)) * 255
    center = size // 2
    radius = size // 4
    
    for y in range(size):
        for x in range(size):
            if (x - center)**2 + (y - center)**2 < radius**2:
                img[y, x] = 50
    
    return img


def create_square_image(size=64):
    """建立方形圖像"""
    img = np.ones((size, size)) * 255
    margin = size // 4
    img[margin:size-margin, margin:size-margin] = 50
    return img


def create_triangle_image(size=64):
    """建立三角形圖像"""
    img = np.ones((size, size)) * 255
    center_x = size // 2
    top_y = size // 4
    bottom_y = 3 * size // 4
    
    for y in range(top_y, bottom_y):
        # 三角形寬度隨 y 增加
        progress = (y - top_y) / (bottom_y - top_y)
        half_width = int(progress * size // 4)
        img[y, center_x - half_width:center_x + half_width] = 50
    
    return img


def create_cross_image(size=64):
    """建立十字形圖像"""
    img = np.ones((size, size)) * 255
    center = size // 2
    arm_width = size // 8
    arm_length = size // 4
    
    # 垂直部分
    img[center-arm_length:center+arm_length, center-arm_width:center+arm_width] = 50
    # 水平部分
    img[center-arm_width:center+arm_width, center-arm_length:center+arm_length] = 50
    
    return img


def add_noise_and_transform(img, noise_std=10, shift_range=5):
    """
    添加噪音和隨機位移
    """
    # 添加高斯噪音
    noisy = img + np.random.randn(*img.shape) * noise_std
    
    # 隨機位移
    shift_y = np.random.randint(-shift_range, shift_range + 1)
    shift_x = np.random.randint(-shift_range, shift_range + 1)
    shifted = np.roll(np.roll(noisy, shift_y, axis=0), shift_x, axis=1)
    
    return np.clip(shifted, 0, 255)


def generate_shape_dataset(n_per_class=100, size=64, noise_std=15):
    """
    生成形狀分類資料集
    
    類別：
    0 - Circle
    1 - Square
    2 - Triangle
    3 - Cross
    """
    creators = [create_circle_image, create_square_image, 
                create_triangle_image, create_cross_image]
    class_names = ['Circle', 'Square', 'Triangle', 'Cross']
    n_classes = len(creators)
    
    X = []
    y = []
    
    for class_idx, creator in enumerate(creators):
        base_img = creator(size)
        
        for _ in range(n_per_class):
            img = add_noise_and_transform(base_img, noise_std)
            X.append(img)
            y.append(class_idx)
    
    X = np.array(X)
    y = np.array(y)
    
    # 打亂
    idx = np.random.permutation(len(y))
    
    return X[idx], y[idx], class_names


# 生成資料集
print("=== 生成形狀分類資料集 ===")

X_images, y_labels, class_names = generate_shape_dataset(n_per_class=150, size=64, noise_std=15)

print(f"資料集大小: {X_images.shape}")
print(f"類別: {class_names}")
print(f"類別分布: {np.bincount(y_labels)}")

# 視覺化樣本
fig, axes = plt.subplots(4, 5, figsize=(12, 10))

for row, class_idx in enumerate(range(4)):
    class_samples = X_images[y_labels == class_idx][:5]
    for col, img in enumerate(class_samples):
        axes[row, col].imshow(img, cmap='gray')
        if col == 0:
            axes[row, col].set_ylabel(class_names[class_idx])
        axes[row, col].axis('off')

plt.suptitle('形狀分類資料集樣本', fontsize=14)
plt.tight_layout()
plt.show()

---

## Part 4: 完整的 HOG + SVM Pipeline

In [ ]:
class HOGSVMClassifier:
    """
    HOG + SVM 影像分類器
    
    Pipeline:
    1. HOG 特徵擷取
    2. SVM 分類
    
    Parameters
    ----------
    n_classes : int
        類別數
    hog_cell_size : int
        HOG cell 大小
    svm_C : float
        SVM 正則化參數
    """
    
    def __init__(self, n_classes, hog_cell_size=8, svm_C=1.0):
        self.n_classes = n_classes
        self.hog = HOGDescriptor(cell_size=hog_cell_size, block_size=2, num_bins=9)
        self.svm = MultiClassSVM(n_classes=n_classes, C=svm_C)
        self.feature_dim = None
    
    def _extract_features(self, images):
        """
        對一批圖像擷取 HOG 特徵
        
        Parameters
        ----------
        images : np.ndarray, shape (N, H, W)
            灰階圖像陣列
        
        Returns
        -------
        features : np.ndarray, shape (N, D)
            HOG 特徵矩陣
        """
        features_list = []
        
        for img in images:
            feat = self.hog.compute(img)
            features_list.append(feat)
        
        return np.array(features_list)
    
    def fit(self, X_images, y, lr=0.001, n_iter=1000, verbose=True):
        """
        訓練分類器
        
        Parameters
        ----------
        X_images : np.ndarray, shape (N, H, W)
            訓練圖像
        y : np.ndarray, shape (N,)
            類別標籤
        """
        if verbose:
            print("Step 1: 擷取 HOG 特徵...")
        
        X_features = self._extract_features(X_images)
        self.feature_dim = X_features.shape[1]
        
        if verbose:
            print(f"  特徵維度: {self.feature_dim}")
            print(f"  訓練樣本數: {len(X_images)}")
            print("\nStep 2: 訓練 SVM...")
        
        self.svm.fit(X_features, y, lr=lr, n_iter=n_iter, verbose=verbose)
        
        if verbose:
            train_acc = self.score(X_images, y)
            print(f"\n訓練完成！訓練準確率: {train_acc:.4f}")
        
        return self
    
    def predict(self, X_images):
        """
        預測類別
        """
        X_features = self._extract_features(X_images)
        return self.svm.predict(X_features)
    
    def score(self, X_images, y):
        """
        計算準確率
        """
        predictions = self.predict(X_images)
        return np.mean(predictions == y)


# 分割訓練集和測試集
def train_test_split(X, y, test_ratio=0.2, seed=42):
    """簡單的訓練/測試集分割"""
    np.random.seed(seed)
    N = len(y)
    n_test = int(N * test_ratio)
    
    idx = np.random.permutation(N)
    test_idx = idx[:n_test]
    train_idx = idx[n_test:]
    
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]


# 訓練模型
print("=== 訓練 HOG + SVM 分類器 ===")

# 分割資料
X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels, test_ratio=0.2)

print(f"訓練集: {len(X_train)} 樣本")
print(f"測試集: {len(X_test)} 樣本\n")

# 建立和訓練分類器
classifier = HOGSVMClassifier(n_classes=4, hog_cell_size=8, svm_C=10.0)
classifier.fit(X_train, y_train, lr=0.0001, n_iter=2000, verbose=True)

# 評估
test_acc = classifier.score(X_test, y_test)
print(f"\n測試準確率: {test_acc:.4f}")

---

## Part 5: 模型評估

In [ ]:
# 混淆矩陣

def compute_confusion_matrix(y_true, y_pred, n_classes):
    """計算混淆矩陣"""
    confusion = np.zeros((n_classes, n_classes), dtype=int)
    for true, pred in zip(y_true.astype(int), y_pred.astype(int)):
        confusion[true, pred] += 1
    return confusion


def plot_confusion_matrix(confusion, class_names):
    """視覺化混淆矩陣"""
    n_classes = len(class_names)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(confusion, cmap='Blues')
    plt.colorbar(im, ax=ax, label='Count')
    
    ax.set_xticks(range(n_classes))
    ax.set_yticks(range(n_classes))
    ax.set_xticklabels(class_names)
    ax.set_yticklabels(class_names)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')
    ax.set_title('Confusion Matrix')
    
    for i in range(n_classes):
        for j in range(n_classes):
            color = 'white' if confusion[i, j] > confusion.max() / 2 else 'black'
            ax.text(j, i, str(confusion[i, j]), ha='center', va='center', color=color, fontsize=12)
    
    plt.tight_layout()
    return fig


def compute_metrics(y_true, y_pred, n_classes, class_names):
    """計算各類別的精確率、召回率、F1"""
    confusion = compute_confusion_matrix(y_true, y_pred, n_classes)
    
    metrics = {}
    
    for i, name in enumerate(class_names):
        tp = confusion[i, i]
        fp = np.sum(confusion[:, i]) - tp
        fn = np.sum(confusion[i, :]) - tp
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        metrics[name] = {'precision': precision, 'recall': recall, 'f1': f1}
    
    return metrics


# 評估測試集
print("=== 模型評估 ===")

y_pred = classifier.predict(X_test)
confusion = compute_confusion_matrix(y_test, y_pred, 4)

print("\n混淆矩陣:")
print(confusion)

# 視覺化
plot_confusion_matrix(confusion, class_names)
plt.show()

# 詳細指標
metrics = compute_metrics(y_test, y_pred, 4, class_names)

print("\n各類別指標:")
print(f"{'Class':<12} {'Precision':>10} {'Recall':>10} {'F1':>10}")
print("-" * 44)
for name, m in metrics.items():
    print(f"{name:<12} {m['precision']:>10.4f} {m['recall']:>10.4f} {m['f1']:>10.4f}")

In [ ]:
# 視覺化一些預測結果

def visualize_predictions(images, y_true, y_pred, class_names, n_show=16):
    """視覺化預測結果"""
    n_cols = 4
    n_rows = (n_show + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 3*n_rows))
    axes = axes.flatten()
    
    for i in range(min(n_show, len(images))):
        axes[i].imshow(images[i], cmap='gray')
        
        true_label = class_names[y_true[i]]
        pred_label = class_names[y_pred[i]]
        
        color = 'green' if y_true[i] == y_pred[i] else 'red'
        axes[i].set_title(f'True: {true_label}\nPred: {pred_label}', color=color)
        axes[i].axis('off')
    
    # 隱藏多餘的 axes
    for i in range(len(images), len(axes)):
        axes[i].axis('off')
    
    plt.suptitle('預測結果（綠色=正確，紅色=錯誤）', fontsize=14)
    plt.tight_layout()
    return fig


# 隨機選擇一些測試樣本
np.random.seed(42)
sample_idx = np.random.choice(len(X_test), 16, replace=False)

visualize_predictions(
    X_test[sample_idx], 
    y_test[sample_idx], 
    y_pred[sample_idx], 
    class_names
)
plt.show()

# 顯示一些錯誤樣本
print("\n錯誤分類的樣本:")
error_idx = np.where(y_test != y_pred)[0]
print(f"共 {len(error_idx)} 個錯誤（共 {len(y_test)} 個測試樣本）")

if len(error_idx) > 0:
    n_show = min(8, len(error_idx))
    visualize_predictions(
        X_test[error_idx[:n_show]], 
        y_test[error_idx[:n_show]], 
        y_pred[error_idx[:n_show]], 
        class_names,
        n_show=n_show
    )
    plt.suptitle('錯誤分類樣本', fontsize=14)
    plt.show()

---

## Part 6: 參數調整實驗

In [ ]:
# 不同 SVM C 值的影響
print("=== SVM C 參數實驗 ===")

C_values = [0.1, 1.0, 10.0, 100.0]
results = []

for C in C_values:
    clf = HOGSVMClassifier(n_classes=4, hog_cell_size=8, svm_C=C)
    clf.fit(X_train, y_train, lr=0.0001, n_iter=1500, verbose=False)
    
    train_acc = clf.score(X_train, y_train)
    test_acc = clf.score(X_test, y_test)
    
    results.append({'C': C, 'train_acc': train_acc, 'test_acc': test_acc})
    print(f"C={C:>6.1f}: Train Acc = {train_acc:.4f}, Test Acc = {test_acc:.4f}")

# 視覺化
plt.figure(figsize=(10, 5))
C_list = [r['C'] for r in results]
train_accs = [r['train_acc'] for r in results]
test_accs = [r['test_acc'] for r in results]

plt.semilogx(C_list, train_accs, 'bo-', linewidth=2, label='Train')
plt.semilogx(C_list, test_accs, 'ro-', linewidth=2, label='Test')
plt.xlabel('SVM C')
plt.ylabel('Accuracy')
plt.title('SVM C 參數 vs 準確率')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# 不同 HOG cell size 的影響
print("=== HOG Cell Size 實驗 ===")

cell_sizes = [4, 8, 16]
results_cell = []

for cell_size in cell_sizes:
    clf = HOGSVMClassifier(n_classes=4, hog_cell_size=cell_size, svm_C=10.0)
    clf.fit(X_train, y_train, lr=0.0001, n_iter=1500, verbose=False)
    
    train_acc = clf.score(X_train, y_train)
    test_acc = clf.score(X_test, y_test)
    
    results_cell.append({
        'cell_size': cell_size, 
        'feature_dim': clf.feature_dim,
        'train_acc': train_acc, 
        'test_acc': test_acc
    })
    print(f"Cell size={cell_size}: Feature dim={clf.feature_dim}, Train={train_acc:.4f}, Test={test_acc:.4f}")

# 視覺化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 準確率
axes[0].bar(np.arange(len(cell_sizes)) - 0.15, [r['train_acc'] for r in results_cell], 0.3, label='Train')
axes[0].bar(np.arange(len(cell_sizes)) + 0.15, [r['test_acc'] for r in results_cell], 0.3, label='Test')
axes[0].set_xticks(range(len(cell_sizes)))
axes[0].set_xticklabels([f'{c}x{c}' for c in cell_sizes])
axes[0].set_xlabel('Cell Size')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Cell Size vs 準確率')
axes[0].legend()

# 特徵維度
axes[1].bar(range(len(cell_sizes)), [r['feature_dim'] for r in results_cell])
axes[1].set_xticks(range(len(cell_sizes)))
axes[1].set_xticklabels([f'{c}x{c}' for c in cell_sizes])
axes[1].set_xlabel('Cell Size')
axes[1].set_ylabel('Feature Dimension')
axes[1].set_title('Cell Size vs 特徵維度')

plt.tight_layout()
plt.show()

print("\n觀察：")
print("- 較小的 cell size 產生更多特徵，捕捉更細節的資訊")
print("- 但可能導致過擬合或計算量增加")

---

## 總結

### 本 Notebook 涵蓋的內容

1. **完整 Pipeline**：
   - HOG 特徵擷取
   - SVM 分類
   - 訓練/測試分割

2. **模型評估**：
   - 準確率
   - 混淆矩陣
   - Precision, Recall, F1

3. **參數調整**：
   - SVM C 參數
   - HOG cell size

### 關鍵要點

1. **特徵工程很重要**：HOG 特徵的設計直接影響分類效果
2. **模型選擇**：SVM 適合高維特徵，因為它關注 margin 而非密度
3. **評估方法**：不只看準確率，要看混淆矩陣了解各類別表現

### Module 4 完成！

恭喜你完成了 Module 4 的所有內容！你已經學會：

- Linear Regression（閉式解和梯度下降）
- Logistic Regression（二分類）
- Softmax Regression（多分類）
- SVM（Hinge Loss）
- K-Means（聚類）
- GMM + EM（進階）
- HOG + SVM（專案整合）

### 下一步

在 **Module 5** 中，我們將學習如何從零實作 CNN！

---

## 練習題

### 練習 1：實作 Cross-Validation

交叉驗證是評估模型泛化能力的重要方法。

**任務**：實作 K-Fold Cross-Validation

**提示**：
1. 將資料分成 K 份
2. 輪流用 K-1 份訓練，1 份驗證
3. 返回平均準確率

In [ ]:
# 練習 1 解答：K-Fold Cross-Validation

def k_fold_cross_validation(X, y, n_classes, k=5, **kwargs):
    """
    K-Fold 交叉驗證
    
    Parameters
    ----------
    X : np.ndarray, shape (N, H, W)
        圖像資料
    y : np.ndarray, shape (N,)
        標籤
    n_classes : int
        類別數
    k : int
        折數
    **kwargs : dict
        傳給分類器的參數
    
    Returns
    -------
    scores : list
        每一折的準確率
    mean_score : float
        平均準確率
    std_score : float
        標準差
    """
    N = len(y)
    indices = np.arange(N)
    np.random.shuffle(indices)
    
    fold_size = N // k
    scores = []
    
    for i in range(k):
        # 分割驗證集和訓練集
        val_start = i * fold_size
        val_end = val_start + fold_size if i < k - 1 else N
        
        val_idx = indices[val_start:val_end]
        train_idx = np.concatenate([indices[:val_start], indices[val_end:]])
        
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # 訓練模型
        clf = HOGSVMClassifier(n_classes=n_classes, **kwargs)
        clf.fit(X_train, y_train, lr=0.0001, n_iter=1000, verbose=False)
        
        # 評估
        score = clf.score(X_val, y_val)
        scores.append(score)
        
        print(f"Fold {i+1}/{k}: Accuracy = {score:.4f}")
    
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    
    return scores, mean_score, std_score


# 測試 5-Fold CV
print("=== 5-Fold Cross-Validation ===")
scores, mean_acc, std_acc = k_fold_cross_validation(
    X_images, y_labels, n_classes=4, k=5, 
    hog_cell_size=8, svm_C=10.0
)

print(f"\n平均準確率: {mean_acc:.4f} ± {std_acc:.4f}")

### 練習 2：Grid Search 參數搜尋

自動搜尋最佳參數組合。

**任務**：實作簡單的 Grid Search，同時搜尋 SVM C 和 HOG cell_size

In [ ]:
# 練習 2 解答：Grid Search

def grid_search(X_train, y_train, X_val, y_val, n_classes, param_grid):
    """
    Grid Search 參數搜尋
    
    Parameters
    ----------
    param_grid : dict
        參數網格，例如 {'svm_C': [1, 10], 'hog_cell_size': [4, 8]}
    
    Returns
    -------
    best_params : dict
        最佳參數
    best_score : float
        最佳分數
    results : list
        所有結果
    """
    from itertools import product
    
    # 生成所有參數組合
    keys = list(param_grid.keys())
    values = list(param_grid.values())
    combinations = list(product(*values))
    
    results = []
    best_score = -1
    best_params = None
    
    print(f"共 {len(combinations)} 種參數組合")
    print("="*50)
    
    for combo in combinations:
        params = dict(zip(keys, combo))
        
        # 訓練
        clf = HOGSVMClassifier(n_classes=n_classes, **params)
        clf.fit(X_train, y_train, lr=0.0001, n_iter=1000, verbose=False)
        
        # 評估
        val_score = clf.score(X_val, y_val)
        train_score = clf.score(X_train, y_train)
        
        results.append({
            'params': params,
            'train_score': train_score,
            'val_score': val_score
        })
        
        print(f"{params} -> Train: {train_score:.4f}, Val: {val_score:.4f}")
        
        if val_score > best_score:
            best_score = val_score
            best_params = params
    
    print("="*50)
    print(f"最佳參數: {best_params}")
    print(f"最佳驗證分數: {best_score:.4f}")
    
    return best_params, best_score, results


# 執行 Grid Search
print("=== Grid Search ===")

# 分割訓練/驗證集
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_ratio=0.2, seed=123)

param_grid = {
    'svm_C': [1.0, 10.0, 100.0],
    'hog_cell_size': [4, 8]
}

best_params, best_score, all_results = grid_search(
    X_tr, y_tr, X_val, y_val, n_classes=4, param_grid=param_grid
)

# 用最佳參數在測試集上評估
print("\n用最佳參數訓練最終模型...")
final_clf = HOGSVMClassifier(n_classes=4, **best_params)
final_clf.fit(X_train, y_train, lr=0.0001, n_iter=1500, verbose=False)
test_score = final_clf.score(X_test, y_test)
print(f"測試集準確率: {test_score:.4f}")

### 練習 3：Data Augmentation

資料增強可以增加訓練資料的多樣性，提高模型泛化能力。

**任務**：實作簡單的資料增強（旋轉、翻轉）並觀察效果

In [ ]:
# 練習 3 解答：Data Augmentation

def rotate_90(image):
    """旋轉 90 度"""
    return np.rot90(image)

def horizontal_flip(image):
    """水平翻轉"""
    return image[:, ::-1]

def vertical_flip(image):
    """垂直翻轉"""
    return image[::-1, :]

def augment_dataset(X, y, augment_funcs):
    """
    對資料集進行增強
    
    Parameters
    ----------
    X : np.ndarray, shape (N, H, W)
    y : np.ndarray, shape (N,)
    augment_funcs : list of callable
        增強函數列表
    
    Returns
    -------
    X_aug : np.ndarray
        增強後的資料（包含原始資料）
    y_aug : np.ndarray
        對應的標籤
    """
    X_list = [X]  # 包含原始資料
    y_list = [y]
    
    for func in augment_funcs:
        X_transformed = np.array([func(img) for img in X])
        X_list.append(X_transformed)
        y_list.append(y)
    
    X_aug = np.concatenate(X_list, axis=0)
    y_aug = np.concatenate(y_list, axis=0)
    
    # 打亂
    idx = np.random.permutation(len(y_aug))
    
    return X_aug[idx], y_aug[idx]


# 測試資料增強效果
print("=== Data Augmentation 實驗 ===")

# 原始資料訓練
print("\n1. 不使用資料增強:")
clf_no_aug = HOGSVMClassifier(n_classes=4, hog_cell_size=8, svm_C=10.0)
clf_no_aug.fit(X_train, y_train, lr=0.0001, n_iter=1000, verbose=False)
score_no_aug = clf_no_aug.score(X_test, y_test)
print(f"   訓練集大小: {len(X_train)}")
print(f"   測試準確率: {score_no_aug:.4f}")

# 使用資料增強
print("\n2. 使用資料增強 (翻轉):")
X_train_aug, y_train_aug = augment_dataset(
    X_train, y_train, 
    [horizontal_flip, vertical_flip]
)

clf_aug = HOGSVMClassifier(n_classes=4, hog_cell_size=8, svm_C=10.0)
clf_aug.fit(X_train_aug, y_train_aug, lr=0.0001, n_iter=1000, verbose=False)
score_aug = clf_aug.score(X_test, y_test)
print(f"   訓練集大小: {len(X_train_aug)} (原本的 3 倍)")
print(f"   測試準確率: {score_aug:.4f}")

print(f"\n準確率變化: {score_no_aug:.4f} -> {score_aug:.4f} ({(score_aug-score_no_aug)*100:+.2f}%)")

In [ ]:
# 視覺化增強效果
fig, axes = plt.subplots(4, 4, figsize=(12, 12))

# 選一個樣本展示
sample_img = X_train[0]

augments = [
    ('Original', lambda x: x),
    ('Horizontal Flip', horizontal_flip),
    ('Vertical Flip', vertical_flip),
    ('Rotate 90°', rotate_90)
]

for row, class_idx in enumerate(range(4)):
    sample = X_train[y_train == class_idx][0]
    
    for col, (name, func) in enumerate(augments):
        axes[row, col].imshow(func(sample), cmap='gray')
        if row == 0:
            axes[row, col].set_title(name)
        if col == 0:
            axes[row, col].set_ylabel(class_names[class_idx])
        axes[row, col].axis('off')

plt.suptitle('Data Augmentation Examples', fontsize=14)
plt.tight_layout()
plt.show()

print("\n觀察：")
print("- 資料增強可以增加訓練資料的多樣性")
print("- 但要注意增強方式要符合實際應用場景")
print("- 例如：對於形狀分類，旋轉可能改變類別（正方形 vs 菱形）")